deep learning model to lassify facial expression from images


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


In [1]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D

from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [14]:
train_dir = 'data/train'
test_dir = 'data/test'
train_data_gen = ImageDataGenerator(rescale = 1/.255)
test_data_gen = ImageDataGenerator(rescale = 1.255)



train_generator =train_data_gen.flow_from_directory(
                            train_dir,
                            target_size =(48,48),
                            batch_size =256,
                            color_mode = "grayscale",
                            class_mode ='categorical')
validation_generator= test_data_gen.flow_from_directory(
                            test_dir,
                            target_size = (48,48),
                            batch_size = 256,
                            color_mode = "grayscale",
                            class_mode= 'categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [15]:
#building the architecture

emotion_model = Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu', input_shape = (48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))

emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))

emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))

emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7, activation='softmax'))




In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 256,
        epochs=50,
        validation_data=validation_generator,
        
                        validation_steps = 7178//256)
                      

Epoch 1/50
 94/112 [========================>.....] - ETA: 9s - loss: 5.0053 - accuracy: 0.1770 

In [5]:
#saving the weights
emotion_model.save_weights('model.h5')

In [ ]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0:'Angry', 1:'Disgusted', 2:'Fearful', 3:'Happy', 4:'Neutral', 5:
                'Sad', 6:'Surprised'}

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bind = cv2.CascadeClassifier(r'C:\Users\shaag\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    num_faces= bind.detectMultiScale(gray_frame, scaleFactor= 1.3, minNeighbors =5)
    
    for x, y, w, h in num_faces:
        cv2.rectangle(frame,(x, y-50), (x+w, y+h+10), (255,0,0),2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex=int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        
    cv2.imshow('Video', cv2.resize(frame, (1200,860), interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) &  0xFF ==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
        

In [5]:
#code fot GUI and Mapping with emojis
import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
global lmain
from keras.preprocessing.image import ImageDataGenerator

emotion_model = Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu', input_shape = (48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))

emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))

emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))

emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('model.h5')


cv2.ocl.setUseOpenCL(False)


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}



emoji_dist={0:".\emojis\\angry.png",1:".\emojis\\disgusted.png",2:".\emojis\\fearful.png",3:".\emojis\\happy.png",4:".\emojis\\neutral.png",5:".\emojis\\sad.png",6:".\emojis\\surprised.png"}


global last_frame1
last_frame1=  np.zeros((480,640,3), dtype = np.uint8)
global  cap1
show_text=[0]
def show_vid():
    
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("Can't open the camera")
    flag1, frame1 = cap1.read()
    frame1= cv2.resize(frame1, (600,500))
    
    
    bind = cv2.CascadeClassifier(r'C:\ProgramData\Anaconda3\envs\tfgpu\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    num_faces= bind.detectMultiScale(gray_frame, scaleFactor = 1.3, minNeighbors= 5)
    
    
    for (x, y , w, h) in num_faces:
        cv2.rectangle(frame1, (x, y -50), (x+w, y+h+10), (255,0,0),2)
        
        roi_gray_frame = gray_frame[y:y +h, x:x+w]
        
        cropped_img =np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)
        prediction = emotion_model.predict(cropped_img)
        maxindex=int(np.argmax(prediction))
        
        cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2, cv2.LINE_AA )
        show_text[0] = maxindex
    if flag1 is None:
        print("Major Error")
    elif flag1:
        global last_frame1 
        last_frame1= frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image = img)
        lmain.imgtk= imgtk
        lmain.configure(image = imgtk)
        lmain.after(10,show_vid)
    if cv2.waitKey(1) &  0xFF==ord('q'):
        exit()
def show_vid2():
    frame2= cv2.imread(emoji_dist[show_text[0]])
    pic2= cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(frame2)
    imgtk2= ImageTk.PhotoImage(image = img2)
    lmain2.imgtk2= imgtk2
    lmain3.configure(text= emotion_dict[show_text[0]], font= 
                     ('arial', 45, 'bold'))
    lmain2.configure(image= imgtk2)
    lmain2.after(10,show_vid2)
    
if __name__ =='__main__':
    root = tk.Tk()
    
    
    heading2 = Label(root, text = "Photo to Emoji", pady = 20, font= ('arial', 20 , 'bold'), bg = 'brown')
    heading2.pack()
    lmain= tk.Label(master= root, padx= 50, bd = 10)
    lmain2 = tk.Label(master= root, bd =10)
    lmain3=tk.Label(master= root,bd = 10, fg = '#CDCDCD', bg = 'brown')
    
    lmain.pack(side= LEFT)
    
    lmain.place(x = 50, y= 250)
    lmain3.pack()
    lmain3.place(x = 960, y= 250)
    lmain2.pack(side= RIGHT)
    lmain2.place(x  =900, y = 350)
    
    root.title('Photo to Emoji')
    root.geometry("1400x900+100+10")
    root['bg'] = 'black'
    exitbutton= Button(root, text= 'Quit', fg=  'red', command = root.destroy, font =('arial', 25, 'bold')).pack(side=BOTTOM)
    show_vid()
    show_vid2()
    root.mainloop()
    




Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\tfgpu\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\ProgramData\Anaconda3\envs\tfgpu\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-5-c7ff12c00a87>", line 52, in show_vid
    cap1 = cv2.VideoCapture(0)
KeyboardInterrupt
